<a href="https://colab.research.google.com/github/chirag-sharma-00/cs182-cv-project/blob/main/inception_merged_10_epochs_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
import glob
import pathlib
import tqdm
import os
import time
import copy
from __future__ import print_function
from __future__ import division
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
from PIL import Image
from google.colab import drive
import shutil

In [ ]:
!rm -rf sample_data

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Load Datasets

In [ ]:
!unzip drive/MyDrive/182-cv-project/data/tiny-imagenet-200.zip
!tar -xvf drive/MyDrive/182-cv-project/data/tiny-imagenet-c.tar
!unzip drive/MyDrive/182-cv-project/data/imagenet-a.zip
!unzip drive/MyDrive/182-cv-project/data/imagenet-o.zip

Streaming output truncated to the last 5000 lines.
  inflating: imagenet-a/n01910747/0.000763_pufferfish _ pufferfish_0.9994343.jpg  
  inflating: imagenet-a/n01910747/0.019548_garter snake _ stingray_0.90012485.jpg  
  inflating: imagenet-a/n01910747/0.077295_sea anemone _ sea anemone_0.63524675.jpg  
  inflating: imagenet-a/n01910747/0.001002_reel _ reel_0.71771824.jpg  
  inflating: imagenet-a/n01910747/0.000555_armadillo _ armadillo_0.9089129.jpg  
  inflating: imagenet-a/n01910747/0.000429_hermit crab _ hermit crab_0.9206918.jpg  
  inflating: imagenet-a/n01910747/0.004313_great egret _ great egret_0.7342569.jpg  
  inflating: imagenet-a/n01910747/0.001530_scorpion _ centipede_0.49828175.jpg  
  inflating: imagenet-a/n01910747/0.009573_flatworm _ flatworm_0.84904796.jpg  
  inflating: imagenet-a/n01910747/0.066160_stingray _ stingray_0.8573044.jpg  
  inflating: imagenet-a/n01910747/0.001533_ant _ ant_0.94848555.jpg  
  inflating: imagenet-a/n01910747/0.002048_pufferfish _ pufferf

In [ ]:
data_dir = pathlib.Path("tiny-imagenet-200")
imagenet_a_path = pathlib.Path('imagenet-a')
imagenet_o_path = pathlib.Path('imagenet-o')
tiny_imagenet_c_path = pathlib.Path('Tiny-ImageNet-C')

In [ ]:
#image_count = len(list(data_dir.glob('**/*.JPEG')))
!rm tiny-imagenet-200/.DS_Store
!rm tiny-imagenet-200/train/.DS_Store
!rm tiny-imagenet-200/val/.DS_Store
!rm tiny-imagenet-200/test/.DS_Store

rm: cannot remove 'tiny-imagenet-200/.DS_Store': No such file or directory
rm: cannot remove 'tiny-imagenet-200/train/.DS_Store': No such file or directory
rm: cannot remove 'tiny-imagenet-200/val/.DS_Store': No such file or directory
rm: cannot remove 'tiny-imagenet-200/test/.DS_Store': No such file or directory


In [ ]:
images = pd.read_csv("tiny-imagenet-200/words.txt", names = ['Id', 'labels'], sep = '\t')

In [ ]:
CLASS_NAMES = np.array([item.name for item in (data_dir / 'train').glob('*')])
CLASS_NAMES.sort()
num_classes = len(CLASS_NAMES)
print(len(CLASS_NAMES)) #should be 200
sum([cls in images['Id'].unique() for cls in CLASS_NAMES])

200


200

In [ ]:
for item in imagenet_a_path.iterdir():
  if item.is_dir() and not item.name in CLASS_NAMES:
    shutil.rmtree(item)
for item in imagenet_o_path.iterdir():
  if item.is_dir() and not item.name in CLASS_NAMES:
    shutil.rmtree(item)
for name in CLASS_NAMES:
  if not pathlib.Path(os.path.join(imagenet_a_path, name)).is_dir():
    os.mkdir(os.path.join(imagenet_a_path, name))
  if not pathlib.Path(os.path.join(imagenet_o_path, name)).is_dir():
    os.mkdir(os.path.join(imagenet_o_path, name))

## Prediction Function

In [ ]:
def predict(dataloaders, model): 
    """
    Run a forward pass (without caching data) for given model and return accuracy
    """
    if torch.cuda.is_available():
      model = model.cuda()

    accuracies = []
    model.eval()
    
    for phase in dataloaders.keys(): 
        counter = 0
        running_corrects = 0
        running_total = 0

        for inputs, labels in dataloaders[phase]: 
            inputs = inputs.to(device)
            labels.data = labels.data.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            counter += 1
            
            running_corrects += torch.sum(preds == labels.data)
            running_total += len(preds)
            
        phase_acc = running_corrects / running_total
        accuracies.append(phase_acc)

    return accuracies
            

## Dataloaders

In [ ]:
input_size = 299
batch_size = 32

In [ ]:
data_transform = transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [ ]:
image_datasets = {
    'imagenet-a': datasets.ImageFolder(imagenet_a_path, data_transform),
    'imagenet-o': datasets.ImageFolder(imagenet_o_path, data_transform),
}
for dir in tiny_imagenet_c_path.iterdir():
  image_datasets["imagenet-c-"+dir.name+"-2"] = datasets.ImageFolder(os.path.join(dir, "2"), data_transform)
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                                   batch_size=batch_size, 
                                                   shuffle=True, num_workers=2) 
                                                   for x in image_datasets.keys()}

## Test All Models

In [ ]:
inception_model = torch.hub.load('pytorch/vision:v0.9.0', "inception_v3", pretrained=True)
inception_aux_in_ftrs = inception_model.AuxLogits.fc.in_features
inception_model.AuxLogits.fc = nn.Linear(inception_aux_in_ftrs, num_classes)
inception_in_ftrs = inception_model.fc.in_features
inception_model.fc = nn.Linear(inception_in_ftrs, num_classes)
resnet_model = torch.hub.load('pytorch/vision:v0.9.0', "resnet50", pretrained=True)
resnet_in_ftrs = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(resnet_in_ftrs, num_classes)

Downloading: "https://github.com/pytorch/vision/archive/v0.9.0.zip" to /root/.cache/torch/hub/v0.9.0.zip
Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [ ]:
models_folder = pathlib.Path("drive/MyDrive/merged_10")

In [ ]:
for item in models_folder.iterdir():
  if item.name.endswith(".pt"):
    if "inception" in item.name:
      model = inception_model
    elif "resnet50" in item.name:
      model = resnet_model
    else:
      print("ARCHITECTURE NOT FOUND")
      break
    print(item)
    model.load_state_dict(torch.load(item))
    accuracies = predict(dataloaders_dict, model)
    for i, key in enumerate(image_datasets.keys()):
      print(key + " accuracy = ", accuracies[i].item())

drive/MyDrive/merged_10/inception_merged_best.pt
imagenet-a accuracy =  0.7152519822120667
imagenet-o accuracy =  0.909274160861969
imagenet-c-zoom_blur-2 accuracy =  0.9910999536514282
imagenet-c-pixelate-2 accuracy =  0.9935999512672424
imagenet-c-snow-2 accuracy =  0.9630999565124512
imagenet-c-impulse_noise-2 accuracy =  0.9660999774932861
imagenet-c-motion_blur-2 accuracy =  0.9904999732971191
imagenet-c-fog-2 accuracy =  0.9911999702453613
imagenet-c-jpeg_compression-2 accuracy =  0.9945999979972839
imagenet-c-shot_noise-2 accuracy =  0.9868999719619751
imagenet-c-contrast-2 accuracy =  0.965399980545044
imagenet-c-gaussian_noise-2 accuracy =  0.9731999635696411
imagenet-c-defocus_blur-2 accuracy =  0.996399998664856
imagenet-c-elastic_transform-2 accuracy =  0.9868999719619751
imagenet-c-glass_blur-2 accuracy =  0.9581999778747559
imagenet-c-frost-2 accuracy =  0.9768999814987183
imagenet-c-brightness-2 accuracy =  0.9914000034332275


In [ ]:
torch.cuda.empty_cache()